In [1]:
!pip install geopandas
!pip install s3fs
# !pip uninstall rtree
!pip install libspatialindex-dev
!pip install rtree
# !pip install install pandas fiona shapely pyproj rtree
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn-whitegrid')
import numpy as np
import pandas as pd
!pip install pygeos
import pygeos
import rtree
from pyproj import CRS
from pyproj import Transformer
import seaborn as sns
sns.set(style="darkgrid")
import scipy as sp
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
!pip install s3fs
!pip install google-cloud-storage 
!pip install --upgrade google-cloud-storage
import gzip
import io
from google.cloud import storage
import pandas as pd


  Using cached geopandas-0.12.2-py3-none-any.whl (1.1 MB)
  Using cached pyproj-3.4.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 71.9 MB/s eta 0:00:0000:0100:01
  Using cached shapely-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.3 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev
  Using cached Rtree-1.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached pygeos-0.14-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)


In [2]:
# create a client object for interacting with the Google Cloud Storage API
client = storage.Client(project="BEAM CORE")

prefix = "output/sfbay/sfbay-pilates-base__2023-03-16_22-27-58_xmg"
bucket_name = "beam-core-outputs"
blob_name = prefix + "/ITERS/it.0/0.events.csv.gz"

# specify the name of the gzipped file you want to create
gz_file = 'example.csv.gz'

# get a reference to the CSV file in the bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)

# download the gzipped file to local storage
blob.download_to_filename(gz_file)

# unzip the file and decode the contents into a text stream
with gzip.open(gz_file, 'rb') as f_in:
    with io.TextIOWrapper(f_in, encoding='utf-8') as f_in_text:
        # read the text stream into a pandas DataFrame
        data = pd.read_csv(f_in_text)
PathTraversal = data[data['type'] == 'PathTraversal']
bikePathTraversal = PathTraversal[PathTraversal['mode'] == 'bike']
sharedbikePathTraversal = bikePathTraversal[bikePathTraversal['vehicle'].str.startswith('bay_wheel')]
crs_4326 = CRS.from_epsg(4326)
crs_26910 = CRS.from_epsg(26910)
transformer = Transformer.from_crs(crs_4326, crs_26910)
sharedbikePathTraversal['coord-x'] = 0.0000000000
sharedbikePathTraversal['coord-y'] = 0.0000000000
sharedbikePathTraversal = sharedbikePathTraversal.reset_index(drop=True)
tuple1 = transformer.transform(sharedbikePathTraversal['startY'], sharedbikePathTraversal['startX'])
sharedbikePathTraversal['coord-x'] = tuple1[0]
sharedbikePathTraversal['coord-y'] = tuple1[1]
taz = gpd.read_file('sfbay-tazs-epsg-26910.shp').to_crs("epsg:26910")
# taz = taz[['objectid','geometry']]
taz['area'] = taz['geometry'].to_crs({'init': 'epsg:26910'}).area
station_centers = gpd.GeoDataFrame(sharedbikePathTraversal,geometry=gpd.points_from_xy(sharedbikePathTraversal['coord-x'],sharedbikePathTraversal['coord-y']), crs='epsg:26910')  
# station_centers = station_centers[['driver','geometry']]
locations = gpd.sjoin(station_centers, taz, how="inner")
usefulLocations = locations[['district', 'taz1454', 'length']]
locations[['district', 'taz1454', 'length']].to_csv('for_length_comparison.csv')
usefulLocations['area'] = ''
usefulLocations.loc[usefulLocations['district'] == 1, 'area']= 'San Francisco'
usefulLocations.loc[usefulLocations['district'] == 2, 'area']= 'San Francisco'
usefulLocations.loc[usefulLocations['district'] == 3, 'area']= 'San Francisco'
usefulLocations.loc[usefulLocations['district'] == 4, 'area']= 'San Francisco'
usefulLocations.loc[usefulLocations['district'] == 19, 'area']= 'Berkeley/Emeryville'
usefulLocations.loc[usefulLocations['district'] == 18, 'area']= 'Oakland'
usefulLocations.loc[usefulLocations['district'] == 9, 'area']= 'San Jose/Santa Clara'
usefulLocations.loc[usefulLocations['district'] == 10, 'area']= 'San Jose/Santa Clara'
usefulLocations.loc[usefulLocations['district'] == 11, 'area']= 'San Jose/Santa Clara'
usefulLocations.loc[usefulLocations['district'] == 12, 'area']= 'San Jose/Santa Clara'
usefulLocations.loc[usefulLocations['district'] == 13, 'area']= 'San Jose/Santa Clara'
(usefulLocations.groupby('area').mean() * 0.000621371).to_csv('averagelength2.csv')
usefulLocations['mile'] = usefulLocations['length'] * 0.000621371
usefulLocations.to_csv('for_length2.csv')
usefulLocations.sort_values(by=['area','mile'],ascending=True).to_csv('for_length2.csv')

In [31]:
usefulLocations.groupby('area').mean().reset_index()[['area','mile']][1:5]

,area,mile
1,Berkeley/Emeryville,1.169589
2,Oakland,0.804122
3,San Francisco,0.637517
4,San Jose/Santa Clara,0.708137
